# Create model with BigML and first submission to Kaggle

In [ ]:
from pandas import read_csv
import bigml.api
from bigml.api import BigML
#api= BigML (project='project/5d8dbde0eba31d7fc0000865')
api=BigML(
source = api.create_source('train.csv')

In [ ]:
api.ok(source)

In [ ]:
dataset_trainfull = api.create_dataset(source)

In [ ]:
api.ok(dataset_trainfull)

In [ ]:
#Split on trainfull 80/20
train_dataset = api.create_dataset(
    dataset_trainfull, {"name": "Train80",
                     "sample_rate": 0.8, "seed": "my seed"})
val = api.create_dataset(
    dataset_trainfull, {"name": "Train20",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

In [ ]:
api.ok(train_dataset)
api.ok(val)

In [ ]:
# Deepnet
model = api.create_deepnet(train_dataset, {"objective_field":"SalePrice"})
api.ok(model)

In [ ]:
evaluation = api.create_evaluation(model, train_dataset)
api.ok(evaluation)

In [ ]:
api.pprint(evaluation['object']['result'])

In [ ]:
# linear regression
linear_regression = api.create_linear_regression(train_dataset)

api.ok(linear_regression)

In [ ]:
evaluation_lr = api.create_evaluation(linear_regression, train_dataset)
api.ok(evaluation_lr)
api.pprint(evaluation_lr['object']['result'])

In [ ]:
#Prediction on deepnet model
batch_prediction = api.create_batch_prediction(model, val, {
    "all_fields": True, "confidence": True, "probabilities": True, "prediction_name": "my_prediction"})
api.ok(batch_prediction)

In [ ]:
api.download_batch_prediction(batch_prediction,
    filename='prediction_val.csv')

In [ ]:
prediction_val = read_csv('prediction_val.csv', index_col=0)

### Model evaluation

Calculation of % Error

In [ ]:
for row in [prediction_val]:
    row['% Error'] = ((row['my_prediction'] - row['SalePrice'])/row['SalePrice'])*100
row

Calculation of Error

In [ ]:
for row in [prediction_val]:
    row['Error'] = row['my_prediction'] - row['SalePrice']
row

Calculation of Absolute Error

In [ ]:
for row in [prediction_val]:
    row['Error_ABS'] = abs(row['Error'])
row

MAE =  mean absolute error

In [ ]:
import numpy as np

target_column = 'Error_ABS'
a = prediction_val[target_column].values

MAE = np.mean(a)

MAE

In [ ]:
for row in [prediction_val]:
    row['Error_ABS_squared'] = row['Error']*row['Error']
row

MSE = mean squared error

In [ ]:
target_column = 'Error_ABS_squared'
b = prediction_val[target_column].values

MSE = np.mean(b)

print(MSE)

In [ ]:
export_csv = prediction_val.to_csv ('prediction_error.csv', header=True)

In [ ]:
#Upload the test
test_source = api.create_source('test.csv')
api.ok(test_source)

In [ ]:
dataset_test = api.create_dataset(test_source)
api.ok(dataset_test)

In [ ]:
# Deepnet on the trainfull dataset
model_2 = api.create_ensemble(dataset_trainfull, {"objective_field":"SalePrice"})
api.ok(model_2)

In [ ]:
batch_prediction_full = api.create_batch_prediction(model_2, dataset_test, {"all_fields": True, "confidence": True, "probabilities": True, "prediction_name": "my_prediction"})
api.ok(batch_prediction_full)

In [ ]:
evaluation_full = api.create_evaluation(model_2, dataset_trainfull)
api.ok(evaluation_full)
api.pprint(evaluation_full['object']['result'])

In [ ]:
api.download_batch_prediction(batch_prediction_full,
    filename='prediction_test.csv')

In [ ]:
# Import the new prediction and remove the columns/replace Id and SalePrice
import pandas as pd
df = pd.read_csv('prediction_test.csv')
shape = df.shape
print(shape)

In [ ]:
list(df.columns.values)

In [ ]:
id_col = df['Id'].values.tolist()
prediction_col = df['my_prediction'].values.tolist()

In [ ]:
import pandas as pd
submission = pd.DataFrame()
submission['Id'] = id_col
submission['SalePrice'] = prediction_col

In [ ]:
submission.to_csv('submission_1.csv', index=False)

In [ ]:
import kaggle
kaggle.api.competition_submit("submission_1.csv", "test", "house-prices-advanced-regression-techniques")

In [ ]:
score = 0.15452
print("Kaggle Score:", score)